In [1]:
import pandas as pd
import csv
import re
import pickle

In [2]:
abbrev_dict = {}

with open('data/Contractions (Clean).csv', mode='r', encoding='utf-8') as infile:
    reader = csv.reader(infile)
    next(reader)
    abbrev_dict = {rows[0]:rows[1] for rows in reader}
    
def expandNotam(msg):
    new_string = re.sub(r'[^a-zA-Z]',' ',str(msg))
    new_msg = ''
    for wrd in new_string.split():        
        try:
            new_msg = new_msg + ' ' + abbrev_dict[wrd]
        except:
            new_msg = new_msg + ' ' + wrd
    return new_msg.lower().strip()

In [3]:
# Read NOTAM Data
df4 = pd.read_csv('data/notam_20201027_pipes_noquotes.csv', 
                  on_bad_lines='skip', 
                  encoding='utf-16', 
                  parse_dates=['POSSIBLE_START_DATE', 'POSSIBLE_END_DATE', 'ISSUE_DATE', 'CANCELED_DATE'],
                  quoting=csv.QUOTE_NONE, 
                  engine="python", 
                  delimiter="|")

# Make MIN_ALT and MAX_ALT numeric
df4['MIN_ALT'] = df4['MIN_ALT'].replace('MSL', 0)
df4['MAX_ALT'] = df4['MAX_ALT'].replace('MSL', 0)
df4['MIN_ALT'] = pd.to_numeric(df4['MIN_ALT'], errors='coerce')
df4['MAX_ALT'] = pd.to_numeric(df4['MAX_ALT'], errors='coerce')

# Make POSSIBLE_START_DATE, POSSIBLE_END_DATE, ISSUE_DATE, and CANCELED_DATE datetime format
df4['POSSIBLE_START_DATE'] = pd.to_datetime(df4['POSSIBLE_START_DATE'], errors = 'coerce', format = '%Y/%m/%d %H:%M:%S')
df4['POSSIBLE_END_DATE'] = pd.to_datetime(df4['POSSIBLE_END_DATE'], errors = 'coerce', format = '%Y/%m/%d %H:%M:%S')
df4['ISSUE_DATE'] = pd.to_datetime(df4['ISSUE_DATE'], errors = 'coerce', format = '%Y/%m/%d %H:%M:%S')
df4['CANCELED_DATE'] = pd.to_datetime(df4['CANCELED_DATE'], errors = 'coerce', format = '%Y/%m/%d %H:%M:%S')

# Replace missing POSSIBLE_START_DATE with CANCELED_DATE and vise versa
df4.loc[df4['CANCELED_DATE'].isna(), 'CANCELED_DATE'] = df4.loc[df4['CANCELED_DATE'].isna(), 'POSSIBLE_END_DATE']
df4.loc[df4['POSSIBLE_END_DATE'].isna(), 'POSSIBLE_END_DATE'] = df4.loc[df4['POSSIBLE_END_DATE'].isna(), 'CANCELED_DATE']

# Get mode of all NOTAM durations
est_days = (df4['POSSIBLE_END_DATE'] - df4['POSSIBLE_START_DATE']).mode()

# Fix remaining missing values of CANCELED_DATE and POSSIBLE_END_DATE by adding the mode to the POSSIBLE_START_DATE
df4.loc[df4['CANCELED_DATE'].isna(), 'CANCELED_DATE'] = df4.loc[df4['CANCELED_DATE'].isna(), 'POSSIBLE_START_DATE'] + pd.to_timedelta(est_days.dt.days[0], unit='D')
df4.loc[df4['POSSIBLE_END_DATE'].isna(), 'POSSIBLE_END_DATE'] = df4.loc[df4['POSSIBLE_END_DATE'].isna(), 'POSSIBLE_START_DATE'] + pd.to_timedelta(est_days.dt.days[0], unit='D')

# Drop NOTAMs without POSSIBLE_END_DATE and POSSIBLE_START_DATE
df4 = df4.dropna( how='all', subset=['POSSIBLE_END_DATE', 'POSSIBLE_START_DATE'])

# Process Msg Text
df4['TEXT'] = df4['TEXT'].apply(expandNotam)

In [4]:
# Check missing values
df4.isna().sum()

NOTAM_REC_ID                    0
FNS_ID                          0
FILENAME                        0
NOTAM_ID                       26
NOTAM_TYPE                 221168
RELATED_NOTAM_ID          1281589
SIMPLE_TEXT                821736
LOC_ID_ACCOUNTABLE_ORG     864049
NOTAM_NUMBER               867227
RELATED_NOTAM_NUMBER      1524160
TEXT                            0
Q_CODE                     639786
Q_CODE_INTERPRETATION      909240
A_CODE                     508084
B_CODE                     666838
C_CODE                     820329
D_CODE                    1480096
E_CODE                        445
F_CODE                     629754
G_CODE                     629952
CLASSIFICATION                 31
POSSIBLE_NOTAM_ID              26
MIN_ALT                    461512
MAX_ALT                    514142
MIN_ALT_REF_TYPE           597577
MAX_ALT_REF_TYPE           665715
POSSIBLE_START_DATE             0
POSSIBLE_END_DATE               0
ISSUE_DATE                     26
CANCELED_DATE 

In [5]:
df4.to_pickle("data/allData.pkl")